In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
condacolab.check()

In [ ]:
!conda create --name nemo_tn python==3.8

In [ ]:
!conda update -n base -c conda-forge conda

In [ ]:
!conda init bash

In [ ]:
%%bash
source activate nemo_tn

In [ ]:
!conda package

In [ ]:
!apt-get install sox libsndfile1 ffmpeg
!pip install wget text-unidecode matplotlib>=3.3.2

# VITS and NeMo

VITS is a neural network that converts text characters into an audio sample. For more details on the model, please refer to Nvidia's [VITS Model Card](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/nemo/models/tts_en_lj_vits), or the original [paper](https://arxiv.org/abs/2106.06103).

VITS like most NeMo models are defined as a LightningModule, allowing for easy training via PyTorch Lightning, and parameterized by a configuration, currently defined via a yaml file and loading using Hydra.

Let's take a look using NeMo's pretrained model and how to use it to generate spectrograms.

# Training

Now that we looked at the VITS model, let's see how to train a VITS Model



In [ ]:
# NeMo's training scripts are stored inside the examples/ folder. Let's grab the vits.py file
# as well as the vits.yaml file
!wget https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/tts/vits.py
!(mkdir -p conf \
  && cd conf \
  && wget https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/tts/conf/vits.yaml \
  && cd ..)

In [ ]:
!wget https://github.com/NVIDIA/NeMo/releases/download/v0.11.0/test_data.tar.gz \
&& mkdir -p tests/data \
&& tar xzf test_data.tar.gz -C tests/data

# Just like ASR, the VITS require .json files to define the training and validation data.
!cat tests/data/asr/an4_val.json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
python manifest_vits.py --data-root /content/drive/MyDrive/ --val-size 0.1 --test-size 0.2

In [ ]:
CUDA_VISIBLE_DEVICES=0 python vits.py model.sample_rate=16000 train_dataset=/content/drive/MyDrive/datasetTTS/train_manifest_vits.json validation_datasets=/content/drive/MyDrive/datasetTTS/val_manifest_vits.json trainer.strategy='ddp_find_unused_parameters_true' trainer.check_val_every_n_epoch=10 +init_from_plt_ckpt=

# Training Data

In order to train VITS, it is highly recommended to obtain high quality speech data with the following properties:
  - Sampling rate of 22050Hz or higher
  - Speech should contain a variety of speech phonemes
  - Audio split into segments of 1-10 seconds
  - Audio segments should not have silence at the beginning and end
  - Audio segments should not contain long silences inside

After obtaining the speech data and splitting into training, validation, and test sections, it is required to construct .json files to tell NeMo where to find these audio files.

The .json files should adhere to the format required by the `nemo.collections.tts.data.dataset.TTSDataset` class. For example, here is a sample .json file

```json
{"audio_filepath": "/path/to/audio1.wav", "text": "the transcription", "duration": 0.82}
{"audio_filepath": "/path/to/audio2.wav", "text": "the other transcription", "duration": 2.1}
...
```
Please note that the duration is in seconds.


## Evaluating VITS

Let's evaluate the quality of the VITS model.

VITS is end-to-end model, so we don't need any additional models to generate audios.

In [ ]:
!wget https://multilangaudiosamples.s3.us-east-2.amazonaws.com/LJ023-0089.wav

In [ ]:
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
import IPython.display as ipd
import numpy as np
import torch
import librosa
import soundfile as sf

target_sr = 22050

audio_path = "./LJ023-0089.wav"
text_raw = "That is not only my accusation."


audio_data, orig_sr = sf.read(audio_path)
if orig_sr != target_sr:
    audio_data = librosa.core.resample(audio_data, orig_sr=orig_sr, target_sr=target_sr)

# Let's double-check that everything matches up!
print(f"Duration (s): {len(audio_data)/target_sr}")
print("Transcript:", text_raw)
ipd.Audio(audio_data, rate=target_sr)

In [ ]:
model = VitsModel.restore_from_checkpoint("tts_en_lj_vits").cpu().eval()

In [ ]:
tokens = model.parse(text_raw)
audio_pred = model.convert_text_to_waveform(tokens=tokens).cpu().detach().numpy()

print("predicted audio")
ipd.Audio(audio_pred, rate=target_sr)

In [ ]:
audio_to_mel = model.audio_to_melspec_processor


len_audio = torch.tensor(len(audio_data)).view(1, -1)

spec_pred, _ = audio_to_mel(torch.tensor(audio_pred).view(1, -1), len_audio)
spec_orig, _ = audio_to_mel(torch.tensor(audio_data).view(1, -1), len_audio)

In [ ]:
fig, ax = plt.subplots(1, 2)

ax[0].imshow(spec_orig[0][0].cpu().detach())
ax[1].imshow(spec_pred[0][0].cpu().detach())

ax[0].set_title('Original spectrogram')
ax[1].set_title('Predicted spectrogram')
fig.show()

In [ ]:
while True:pass